
# Рынок заведений общественного питания Москвы

Учебная работа. Самостоятельный исследовательский проект в рамках курса "Аналитик данных" Яндекс.Практикума.  
Спринт 9: "Как рассказать историю с помощью данных"

Без output (не сохранился файл с данными)
 
## Общая информация о проекте
### Описание задания
Есть открытые данные о заведениях общественного питания в Москве. Нужно проанализировать текущее положение на рынке общественного питания в Москве и дать рекомендации о том, какой тип заведения, количество посадочных мест и район расположения лучше выбрать инвесторам.

###  Цель
Проанализировать от крытые данные и сформулировать рекомендации о наиболее выгодном расположении, типе и размере кафе.

### Задачи
1. Подготовить данные к работе (проверка типа данных в столбцах, наличия пропусков, ошибок и дубликатов)
2. Изучить характеристики сетевых и несетевых заведений общественного питания.
3. Сравнить сетевые и несетевые заведения по количеству посадочных мест.
4. Выделить районаы с наибольшим количеством объектов общественного питания.
5. Выделить районаы с наименьшим количеством объектов общественного питания.
6. Изучить закономерности распределения количества посадочных мест в заведениях из наиболее популярных районов.


###  Данные
Данные о заведениях общественного питания получены в виде файла `rest_data.csv`.

###  Описание данных

**Структура файла `rest_data.csv`**  
 - `id` — идентификатор объекта;
 - `object_name` — название объекта общественного питания;
 - `chain` — сетевой ресторан;
 - `object_type` — тип объекта общественного питания;
 - `address` — адрес;
 - `number` — количество посадочных мест.


## Загрузка и обзор данных

Загружаем библиотеки:

In [ ]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
import datetime as dt
import matplotlib.pyplot as plt
from scipy import stats as st
import math
import seaborn as sns
import re
import pprint

import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")

Создаём библиотеку стилей:

In [ ]:
pd.set_option('max_colwidth', 500)
sns.set_style("ticks")

class Color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Загружаем файл с данными:

In [ ]:
data = pd.read_csv('/datasets/rest_data.csv')

### Обзор и подготовка данных
Выведем первые 5 строк таблицы `data`:

In [ ]:
display(data.head())

Проверим данные о столбцах:

In [ ]:
data.info()

Пропусков нет, тип данных в столбцах правильный. Проверим данные на наличие дубликатов:

In [ ]:
print('Дублирующихся строк:', data.duplicated().sum())

Проверим данные в столбцах на наличие ошибок, создадим необходимы столбцы (выделим в отдельный столбец данные об улицах).  

#### Идентификатор объекта общественного питания

В столбце `id` числовые значения, выведем описательную статистику и посчитаем дублирующиеся значения:

In [ ]:
print(data['id'].duplicated().sum())
data['id'].describe()

Дубликатов в столбце `id` нет, но номера распределены очень странно и явно не по порядку (в данных 15 тысяч строк, а номера `id` - от 838 до 223 тысяч). Проверим, есть ли в данных дубликаты, если не учитывать столбец `id`.

In [ ]:
print('Дублирующихся строк (без столбца `id`):', 
      data[['object_name', 'chain', 'object_type', 'address', 'number']].duplicated().sum())

Дублирующие строки нашлись. Столбец `id` не содержит ценной информации, его можно удалить.  
Удаляем столбец `id`, удаляем дубликаты и делаем проверку.

In [ ]:
data = data.drop('id', axis=1).drop_duplicates()
print('Дублирующихся строк:', data.duplicated().sum())

**Вывод:** столбец `id` удалён.

#### Название объекта общественного питания

Проверим на наличие ошибок столбец `object_name`.

In [ ]:
print(data['object_name'].unique())
data['object_name'].describe()

Из 15284 значений 10393 - уникальные, самое частотное, "столовая", встречается 267 раз. Не удивительно, что самое частотное название - это "Столовая", ведь столовые есть везде (и в школах, и в вузах, и в офисах), и не требуют отдельного названия.    
Всречается написание капслоком и кавычки, которые могут приводить к возникновению дубликатов. 

Для начала изменим регистр во всех названиях и проверим данные на дубликаты.

In [ ]:
data['object_name'] = data['object_name'].str.lower()

print(data['object_name'].unique())
print('Дублирующихся строк:', data.duplicated().sum())
data['object_name'].describe()

Найдено три дубликата, а уникальных названий стало меньше.  
Лемматизируем названия и отсортируем леммы по алфавиту при помощи функции. Результат сохраним в столбце `lemmas`.

In [ ]:
def lemmas(str):
    lemmas = m.lemmatize(str)
    lemmas.sort(reverse=True)
    return lemmas

print('Проверка работы функции:', lemmas('мята lounge октябрьская'))
print('Проверка работы функции:', lemmas('лаундж-бар мята lounge'))

data['lemmas'] = data['object_name'].apply(lemmas)

Удалим кавычки и другие лишние символы из столбца `lemmas` для удобства восприятия и сохраним лемматизированное название объекта в отдельный столбец.

In [ ]:
def join_lemmas(list):
    joined_lemmas = ' '.join(list)
    joined_lemmas = joined_lemmas.replace('«', '').replace('»', '').replace('\n', '').replace(
        '   ', ' ').replace('-', '').replace('  ', ' ').replace('  ', ' ')
    return joined_lemmas
    
data['lemmatized_object_name'] = data['lemmas'].apply(join_lemmas)
print(data['lemmatized_object_name'].unique())
print()
print('Дублирующихся строк:', data[['chain', 'object_type', 'address', 'number', 
       'lemmatized_object_name']].duplicated().sum())
data['lemmatized_object_name'].describe()

Нашлась ещё одна дублирующаяся строка, а количество уникальных названий ещё немного уменьшилось.Удалим дублирующиеся строки и сделаем проверку. Исходные названия сохраним в переменную `original_name` и добавим в получившуюся таблицу.

In [ ]:
original_name = data['object_name']
print('Количество строк до удаления дубликатов:', len(data))
data = data[['chain', 'object_type', 'address', 'number', 
       'lemmatized_object_name']].drop_duplicates()
print('Количество строк после удаления дубликатов:', len(data))
data = data.join(original_name)
data = data.rename(columns={'object_name': 'original_name'})
print('Количество строк после присоединения столбца с исходными названиями', len(data))

print('Дублирующихся строк:', data[['chain', 'object_type', 'address', 'number', 
       'lemmatized_object_name']].duplicated().sum())

Удалим временный технический столбец `lemmatized_object_name`

In [ ]:
data = data.drop('lemmatized_object_name', axis=1)
data.columns

**Вывод:** данные о названиях заведений подготовлены.

#### Сетевой или нет
Проверим значения в столбце `chain`:

In [ ]:
data['chain'].describe()

Встолбце `chain` всего два уникальных значения (да и нет). Несетевых заведений гораздо больше (12 тысяч из 15).  
Заменим логические значения "да" и "нет" на смысловые "сетевой" и "несетевой" и сделаем проверку

In [ ]:
data['chain'] = data['chain'].str.replace('да', 'сетевой')
data['chain'] = data['chain'].str.replace('нет', 'несетевой')

print(data['chain'].unique())

**Вывод:** данные в столбце `chain` подготовлены.


#### Тип объекта общественного питания
Проверим корректность данных в столбце `object_type`:

In [ ]:
print(data['object_type'].unique())
data['object_type'].describe()

Всего девять типов, ошибок нет. Правда, не очень понятно, чем кафетерий отличается от буфета и что именно имеется в виду под "предприятием быстрого обслуживания". Посмотрим, какие заведения попадают в каждый из типов.

In [ ]:
data.columns

In [ ]:
types = data.pivot_table(
    index='object_type', 
    columns='chain', 
    values='original_name', 
    aggfunc='first'
)

display(types)

Разделение на типы питания явно очень условно и сделано по разным основаниям. "Суши wok" и "Суши вок", которые с большой вероятностью относятся к одной сети, попадают сразу и в закусочные, и в предприятия быстрого обслуживания. "Грабли" - скорее ресторан, но в наших данных записан, как столовая. "Кофе с собой" логичнее было бы отнести к предприятиям быстрого обслуживания. 

**Вывод:** данные в столбце `object_type` подготовлены, но при анализе стоит учитывать, что, скорее всего, типы выделены условно и могут пересекаться.

#### Адрес
Проверим данные в столбце адрес

In [ ]:
print(data['address'].unique())

Выделим в отдельный столбец улицу.  
Составим список слов, обозначающих улицы и применим к столбцу `address` регулярное выражение. Результат сохраним в столбец `street`.

In [ ]:
words = ['улица','ул','переулок','шоссе','проспект','площадь','проезд',
         'село','аллея','бульвар','набережная','тупик','линия', 'квартал', 'микрорайон']

street_pattern = r".*,\s*\b([^,]*?(?:{})\b[^,]*)[,$]+".format("|".join(words))

data['street'] = data['address'].str.extract(street_pattern)

Сделаем проверку: распечатаем уникальные значения сталбца `street` и первые 5 строк таблицы `data`.

In [ ]:
unique = data['street'].unique()
display(unique)

In [ ]:
display(len(data.loc[data.loc[:, 'street'].isna()]))
display(data.loc[data.loc[:, 'street'].isna()].head())

Не удалось извлечь названия улиц в случаях, когда не указан город (строки 859-860) или в адресе совсем нет улицы (например, город Зеленоград, корпус 436). Во втором случае пропуск будет верным вариантом, а в первом адрес не получилось извлечь из-за того, что названию улицы не предшествует запятая. Применим отредактированный вариант регулярного выражения:

In [ ]:
def pat2(row):
    street_pattern2 = r"(.*\s*\b[^,]*?(?:{})\b[^,]*)[,$]+".format("|".join(words))
    
    search_pat = re.search(street_pattern2, row['address'])
    if pd.isna(row['street']):
        if pd.isnull(search_pat):
            return 'Нет'
        else:
            return search_pat.group()
            print(search_pat.group())
    else:
        return row['street']

In [ ]:
data['street'] = data.apply(pat2, axis=1)
display(len(data.loc[data.loc[:, 'street'].isna()]))

В названиях могут встречаться скрытые дубликаты с разным написанием. Заменим в столбце `street` букву "ё" на "е", удалим лишние запятые и поменяем регистр:

In [ ]:
data['street'] = data['street'].str.replace(',', '').str.replace('ё', 'е')
data['street'] = data['street'].str.lower()

**Вывод:** данные об адресе подготовлены.

#### Количество посадочных мест
Проверим данные в столбце `number`:

In [ ]:
data['number'].describe()

Выглядит вполне правдоподобно кроме максимального значения в 1700 посадочных мест. Бывают ли такие огромные заведения общественного питания? Отсортируем таблицу по убыванию количества посадочных мест и выведем первые 10 строк:

In [ ]:
display(data.sort_values(by='number', ascending=False).head(10))

Похоже, это не ошибка, и такие огромные рестораны действительно бывают. 
**Вывод:** данные о количестве посадочных мест готовы к работе.

#### Вывод 
Данные подготовлены к работе. Проверены типы данных в столбцах, удалёны дубликаты, данные проверены на наличие пропусков и ошибок. Выделены в отдельный столбец данные об улицах.

## Анализ данных
### Типы объектов общественного питания

Посчитаем и построим график количества объектов общественного питания по типам:

In [ ]:
rest_types = data.groupby(['object_type'])[
    'original_name'].count().reset_index().sort_values(by='original_name', ascending=False)

sns.barplot(y='object_type', x='original_name', data=rest_types).set(
    title='Количество заведений общественного питания по типам \n',
    xlabel='Количество заведений',
    ylabel='Тип заведения')
plt.show()

Чаще всего встречается кафе, реже столовые, рестораны и предприятия быстрого обслуживания. Меньше всего по количеству буфетов, баров, закусочных и отделов кулинарий.

#### Вывод
В Москве больше всего заведений типа кафе, меньше столовые, рестораны и предприятия быстрого обслуживания. Меньше всего по количеству буфетов, баров, закусочных и отделов кулинарий.

### Сетевые и несетевые заведения

Посмотрим, сколько заведений - сетевые:

In [ ]:
rest_chain = data.groupby(['chain'])['original_name'].count().reset_index()
sns.barplot(x='chain', y='original_name', data=rest_chain).set(
    title='Количество сетевых и несетевых заведений в Москве \n',
    ylabel='Количество заведений',
    xlabel='Тип заведения')
plt.show()

Сетевых заведених гораздо меньше. Посмотрим, как они распределены по типам заведений:

In [ ]:
rest_chain_type = data.groupby(['object_type', 'chain'])[
    'original_name'].count().reset_index().sort_values(by='original_name', ascending=False)
plt.figure(figsize=(8, 5))
sns.barplot(y='object_type', x='original_name', data=rest_chain_type, hue='chain').set(
    title='Количество заведений общественного питания по типам \n',
    xlabel='Количество заведений',
    ylabel='Тип заведения')

plt.show()

Несетевых заведений больше во всех категориях заведений общественного питания, кроме предприятий быстрого обслуживания. Достаточно большая доля сетевых заведених среди кафе, ресторанов, а столовых, буфетов и баров почти нет.

##### Вывод
В Москве преобладают несетевые заведения общественного питания. Сетевых заведений больше только среди предприятий быстрого обслуживания. Достаточно большая доля сетевых заведених среди кафе, ресторанов.

### Количество посадочных мест в сетевых и несетевых заведениях

Посчитаем медианное количество посадочных мест в сетевых и несетевых заведениях и посроим график.

In [ ]:
sns.barplot(y='chain', x='number', data=data, errwidth = 0).set(
    title='Количество посадочных мест в сетевых и несетевых \n заведениях общественного питания \n',
    xlabel='Количество посадочных мест',
    ylabel='Тип заведения')
plt.show()

Посроим график количества посадочных мест в сетевых и несетевых заведениях с учётом типа заведения.

In [ ]:
plt.figure(figsize=(8, 7))
sns.boxplot(y="object_type", x="number", data=data, hue='chain', fliersize = 0).set(
    title='Распределение количества посадочных мест  \n в зависимости от типа заведения',
    xlabel='Количество посадочных мест',
    ylabel='Тип заведения')
plt.xlim(-10, 400)

plt.show()

**Столовые и буфеты:** разброс количества посадочных мест в сетевых столовых и буфетах очень маленький (скорее всего, такие встречаются очень редко). Посадочных мест в столовых больше, чем в буфетах, а разброс количества посадочных мест в столовых больше, чем во всех других типах заведений.

**Закусочные и предприятия быстрого обслуживания, магазины (отделы кулинарий) и кафетерии:** Разброс количества посадочных мест в закусочных и предприятиях быстрого обслуживания больше в сетевых заведениях, посадочных мест больше в предприятиях быстрого обслуживания. 

**Бары:** в сетевых барах больше посадочных мест, чем в несетевых.

**Кафе и рестораны:** в ресторанах разброс и количество посадочных мест гораздо больше, чем в кафе. Сетевые и несетевые заведения различаются не сильно.

#### Вывод

Больше всего посадочных мест обычно в столовых, ресторанах и сетевых барах.  
Меньше всего посадочных мест в закусочных, кафетериях и отделах кулинарий.  
В предприятиях быстрого обслуживания и барах в сетевых заведениях больше посадочных мест, чем в несетевых.  
В сетевых столовых и буфетах посадочных мест гораздо меньше, чем в несетевых.

### Данные о районе

Загрузим таблицу с данными о том, в каком районе находятся улицы Москвы.

In [ ]:
area = pd.read_csv(
    'https://frs.noosphere.ru/xmlui/bitstream/handle/20.500.11925/714058/mosgaz-streets.csv?sequence=1&isAllowed=y')
display(area.head())

В таблице есть данные не только о районе, но и об округе, это хорошо.  

Заменим в названиях улиц букву "ё" на "е" и изменим регистр на строчные буквы. Удалим столбец `areaid`, он нам не пригодится.

In [ ]:
area['streetname'] = area['streetname'].str.replace(',', '').str.replace('ё', 'е')
area['streetname'] = area['streetname'].str.lower()
area = area.drop('areaid', axis=1)
area.columns

Присоединим к таблице data данные о районах и выведем первые пять строк получившейся таблицы:

In [ ]:
data_joined = data.merge(area, how='left', left_on='street', right_on='streetname')
display(data_joined.head(2))

Проверяем, есть ли улицы, которые не удалось соотнести с районом. Сгруппируем проблемные данные по улицам, для каждой улицы выведем пример полного адреса (при помощи функции `first`) и количество заведений.

In [ ]:
problem_streets = data_joined.loc[data_joined['okrug'].isna() & (data_joined['street'] != 'нет')]

display(problem_streets.groupby('street').agg(
    {'address' : 'first', 'original_name' : 'count'}).sort_values(
    by='original_name', ascending=False).head(25))

В большинстве строк, в которых не удалось соотнести данные о районах, заведение находится за пределами МКАД и данных о районе нет (это города Троик, Московский, поселение Сосенское и т.д.). Заведения за пределами МКАД не являются предметом нашего интереса, поэтому их можно не включать в анализ.  

Также нет данных о некоторых улицах внутри МКАДа (например, о Бесединском шоссе и Берсеневском переулке).   

Ошибка в распознавании улицы Мичуринский проспект. В файле она есть, но с полным названием ("улица Мичуринский Проспект, Олимпийская Деревня"), можно сделать замену.  
В загруженном файле нет 3-его Митинского переулка, но есть 3-й Митинский проезд (и 1-й и 2-й Митинские переулки), скорее всего, имеется в виду одна и та же улица, можно сделать замену.  

Улицы с количеством заведений меньше пяти, скорее всего, тоже большей частью находятся за МКАД. Если есть такие, которые находятся в пределах МКАД, то их немного, и они не сильно повлияют на выводы, ими можно пренебречь.

Исправляем ошибки и делаем ещё одну проверку.

Сделаем замену в строках с Олимпийской деревней и 3-м Митинским переулком:

In [ ]:
data['street'] = data['street'].str.replace(
    'улица мичуринский проспект', 'улица Мичуринский Проспект Олимпийская Деревня').str.lower()
data['street'] = data['street'].str.replace('3-й митинский переулок', '3-й Митинский проезд').str.lower()

Добавим данные о Берсеневском переулке и Бесединском шоссе:

In [ ]:
def missed_streets(row, street, column, result):
    if row['street'] == street:
        return result
    else:
        return row[column]
    
data_joined = data.merge(area, how='left', left_on='street', right_on='streetname')

In [ ]:
"""Исправляем данные о Берсеневском переулке"""

data_joined['okrug'] = data_joined.apply(missed_streets, axis=1, args=('берсеневский переулок', 'okrug', 'ЦАО',))
data_joined['area'] = data_joined.apply(missed_streets, axis=1, args=('берсеневский переулок', 'area', 'Район Якиманка',))

"""Исправляем данные о Бесединском шоссе"""

data_joined['okrug'] = data_joined.apply(missed_streets, axis=1, args=('бесединское шоссе', 'okrug', 'ЮВАО',))
data_joined['area'] = data_joined.apply(missed_streets, axis=1, args=('бесединское шоссе', 'area', 'Район Капотня',))

Делаем проверку (выводим по две строки для исправленных улиц, для одной улицы, по которой изначально были правильные данные и одной, по которой нет данных об улице):

In [ ]:
display(data_joined.loc[data_joined['street'] == 'берсеневский переулок'].head(2))
display(data_joined.loc[data_joined['street'] == 'бесединское шоссе'].head(2))
display(data_joined.loc[data_joined['street'] == 'юбилейная улица'].head(2))
display(data_joined.loc[data_joined['street'] == 'улица пречистенка'].head(2))

Ещё раз выводим проблемные улицы:

In [ ]:
problem_streets = data_joined.loc[data_joined['okrug'].isna() & (data_joined['street'] != 'нет')]

display(problem_streets.groupby('street').agg(
    {'address' : 'first', 'original_name' : 'count'}).sort_values(
    by='original_name', ascending=False).head(15))

#### Вывод

В таблицу добавлены данные о районах и округах для улиц, находящихся внутри МКАД.

### Топ-10 улиц по количеству заведений общественного питания.

Сгруппируем данные по улице и выведем первые 10 значений в порядке убывания:

In [ ]:
data_street = data_joined.groupby('street').agg(
    {'original_name' : 'count', 
     'okrug' : ['unique', 'nunique'], 
    'area' : ['unique', 'nunique'], }).reset_index()

data_street.columns = ['street', 'n_objects', 'district', 'n_districts', 'area', 'n_areas']

top_streets = data_street.sort_values(by ='n_objects', ascending=False).head(10)
pd.set_option('max_colwidth', 500)
display(top_streets)

In [ ]:
sns.barplot(y='street', x='n_objects', data=top_streets).set(
    title='Топ-10 улиц по количеству заведений общественного питания \n',
    xlabel='Количество заведений',
    ylabel='Улица')
plt.show()

В Топ-10 попали длинные магистральные улицы и шоссе, ведущие от центра за пределы города. Они проходят через несколько районов, а иногда даже округов. Вероятно, именно здесь находятся крупные транспортные узлы, метро, и эти улицы пропускают через себя большое количество людей.   
С другой стороны, эти улицы - самые длинные. Вполне возможно, что, чем длиннее улица, тем больше на ней может разместиться заведений общественного питания (а на маленькой улице, даже если она очень популярна, физически заведений может поместиться меньше). 

При оценке привлекательности улицы для ресторанного бизнеса было бы интересно оценить не только количество заведений на улице, но и плотность. Попробуем примерно оценить плотность, разделив количество заведений общественного питания на количество районов, через которые проходит эта улица. Эта метрика будет весьма приблизительной, так как мы не знаем, равномерно ли распределены заведения по улице или нет. Но, при допущении о том, что заведения распределены равномерно, она далжна позволить нам сравнивать улицы, пересекающие несколько районов, и улицы, полностью располагающиеся в границах одного района.

In [ ]:
data_street['nobjects_per_area'] = data_street['n_objects'] / data_street['n_areas']
top_streets_per_area = data_street[data_street['n_areas'] != 0].sort_values(by ='nobjects_per_area', ascending=False).head(10)
display(top_streets_per_area)

In [ ]:
sns.barplot(y='street', x='nobjects_per_area', data=top_streets_per_area).set(
    title='Топ-10 улиц по количеству заведений общественного питания (с учётом длины) \n',
    xlabel='Количество заведений',
    ylabel='Улица')
plt.show()

Даже при учёте длины улицы лидерами остаются улицы-магистрали. Но появились и новые названия: очень много заведений на Пресненской набережной, хотя она довольно короткая (находится полностью в пределах одного района), но в центре, и почему-то Кировоградская улица (это странно, потому что Кировоградская улица проходит через спальные районы и там нет крупных транспортных узлов).  

Составим топ-список популярных улиц, полученных первым и вторым способом (с корректировкой с учётом длины и без), сохраним его в переменной `top_streets`.

In [ ]:
top_street_list = pd.Series(top_streets['street'].unique())
top_street_list_2 = pd.Series(top_streets_per_area['street'].unique())

top_streets = top_street_list.append(top_street_list_2).drop_duplicates().reset_index(drop=True)
print(top_streets)

Список самых популярных улиц составлен. 

**Топ-10 районов**  

Сгруппируем таблицу по районам, чтобы посмотреть, какие районы более популярны у владельцев заведений общественного питания.  
В результате применения группировки в столбцах `okrug` и `area` получились списки. Для дальнейшей группировки преобразуем списки в текст.

In [ ]:
def join_string(series):
    return (', '.join(map(str, series)))

In [ ]:
data_street['district'] = data_street['district'].apply(join_string)
data_street['area'] = data_street['area'].apply(join_string)

display(data_street.head(2))

Сгруппируем данные по районам и выведем Топ-10 районов по количеству заведений общественного питания. 

In [ ]:
data_area = data_street.groupby('area').agg(
    {'street' : 'first',
     'n_objects' : 'sum', 
    'district' : 'first',
     'n_areas' : 'first'
    }).reset_index()

display(data_area.sort_values(
    by ='n_objects', ascending=False).head(10))

В лидерах по прежнему остаются магистральные улицы. Ограничим выдачу теми улицами, которые располягаются в пределах одного района. Это не полные данные по району, так как отсюда исключены все улицы, проходящие через два и более районов (крупные улицы с большим количеством заведений общественного питания). Но такой способ позволит оценить привлекательность маленьких локальных улиц для ресторанного бизнеса.

In [ ]:
top_areas_data = data_area.loc[data_area.loc[:, 'n_areas'] == 1].sort_values(
    by ='n_objects', ascending=False).head(10)

top_areas = top_areas_data['area'].unique()
display(top_areas_data)

Почти все районы из Топ-10 находятся в Центре, что логично. Только три района выбиваются: Даниловский (административно он не относится к ЦАО, но по смыслу очень близок), Хорошёвский, который считается весьма элитным районом, и Митино - по сути полноценный город. 

#### Вывод
Больше всего заведений общественного питания расположено на магистральных улицах, ведущих из центра за пределы города: 
 - проспект Мира,
 - Варшавское шоссе
 - Ленинский проспект
 - Профсоюзная улица
 - Дмитровское шоссе
 - Ленинградский проспект
 - проспект Вернадского
 - Ленинградское шоссе
 - Каширское шоссе
 - Волгоградский проспект
 - Пресненская набережная
 - Кутузовский проспект
 - Кировоградская улица   
 
Большое количество заведений общественного питания на магистральных улицах (магистральные улицы остаются самыми популярными даже после коррекции на длину улицы). Скорее всего, это связано с тем, что на них находятся крупные транспортные узлы, через которые проходит много людей.
 
На маленьких улицах, располагающихся в границах одного района, больше всего заведений общественного питания в центре (ЦАО).

### Улицы с одним заведением общественного питания

Найдём улицы с одним заведением общественного питания:

In [ ]:
the_only_object_streets = data_street[data_street['n_objects'] == 1]
print(len(the_only_object_streets))
display(the_only_object_streets.head())

Всего найдено 509 улиц, на которых находится всего одно заведение общесвенного питания. Сгруппируем их по районам и выведем районы с наибольшим количеством улиц с единственным заведением общественного питания:

In [ ]:
data_area_only_object_streets = the_only_object_streets.groupby('area').agg(
    {'street' : 'count',
    'district' : 'first',
    }).reset_index()

display(data_area_only_object_streets.sort_values(
    by ='street', ascending=False).head(10))

In [ ]:
data_area_only_object_streets = the_only_object_streets.groupby('area').agg(
    {'street' : 'count', 'district' : 'first'})
data_area_all_objects_streets = data_street.loc[data_street.loc[:, 'n_areas'] == 1].groupby('area').agg(
    {'street' : 'count'})
data_area_all_objects_streets.columns = ['all_streets_with_objects']

data_area_ = data_area_only_object_streets.join(data_area_all_objects_streets).reset_index()
data_area_['proportion'] = data_area_['street'] / data_area_['all_streets_with_objects']
display(data_area_.sort_values(by ='proportion', ascending=False).head(10).round(2))

Построим график доли улиц с единственным заведением общественного питания от всех заведений района на маленьких улицах.

In [ ]:
def join_columns(row):
    str1 = row['area']
    str2 = row['district']
    
    result = (str1 + ', ' + str2)
    return result

In [ ]:
data_area_['area_district'] = data_area_.apply(join_columns, axis=1)
plt.figure(figsize=(6, 20))
sns.barplot(y='area_district', x='proportion', data=data_area_.sort_values(by ='proportion', ascending=False).head(50)).set(
    title='Доля улиц с одним заведением общественного питания \n от количества локальных улиц района (на которых есть хотя бы одно заведение). \n',
    xlabel='Доля улиц с единственным заведением',
    ylabel='Район')

plt.show()

Больше всего доля улиц с единственным заведением общественного питания (больше 50%) в периферийных спальных районах. Скорее всего, здесь небольшой спрос на ресторанные услуги, общее количество заведений общественного питания очень небольшое, и достаточно одного такого заведения на большую площадь.

Найдём долю улиц с единственным заведением общественного питания от всех локальных улиц округа, на которых есть хотя бы одно заведение.

In [ ]:
data_district_only_object_streets = the_only_object_streets.groupby('district').agg(
    {'street' : 'count'})
data_district_all_objects_streets = data_street.loc[data_street.loc[:, 'n_districts'] == 1].groupby('district').agg(
    {'street' : 'count'})
data_district_all_objects_streets.columns = ['all_streets_with_objects']

data_district = data_district_only_object_streets.join(data_district_all_objects_streets).reset_index()
data_district['proportion'] = data_district['street'] / data_district['all_streets_with_objects']
# display(data_district.sort_values(by ='proportion', ascending=False).head(10).round(2))

# plt.figure(figsize=(6, 20))
sns.barplot(y='district', x='proportion', data=data_district.sort_values(by ='proportion', ascending=False)).set(
    title='Доля улиц с одним заведением общественного питания \n от количества локальных улиц округа (на которых есть хотя бы одно заведение). \n',
    xlabel='Доля улиц с единственным заведением',
    ylabel='Округ')

plt.show()

Доля локальных улиц с единственным заведением общественного питания не сильно различается по округам (в семи округах, где она наибольшая, она колеблется от 27% до 34%).  
Меньше всего доля улиц с единственным заведением общественного питания в ЮЗАО, Зеленограде и ЮАО. 

##### Вывод
Больше всего улиц с единственным заведением общественного питания находится в центре, однако доля таких улиц от общего количества улиц (на которых есть хотя бы одно заведение общественного питания) больше всего в периферийных спальных районах. При этом доля улиц с единственным заведением сильно различается по районами внутри одного округа. Вероятно, это связано с тем, что в спальных районах небольшой спрос на услуги кафе и ресторанов, и общее количество таких заведений небольшое, а ближе к транспортным узлам или другим точкам притяжения посетителей, спрос больше.

### Распределение количества посадочных мест в зависимости от количества объектов общественного питания на улице

Посмотрим на количество посадочных мест для улиц из списка `top_streets`:

In [ ]:
plt.figure(figsize=(15, 7))
sns.boxplot(y="street", x="number", data=data.query('street in @top_streets'), fliersize = 4).set(
    title='Количество посадочных мест на улицах с наибольшим количеством \n заведений общественного питания',
    xlabel='Количество посадочных мест',
    ylabel='Улица')

plt.show()

Завезения с самым большим количеством посадочных мест находятся на Кутузовском проспекте и проспекте Вернадского (и тот, и другой - достаточно дорогие районы). Среднее количество посадочных мест на всех улицах небольшое: до 100.    
Спрячем выбросы и добавим подкатегорию: сетевое заведение или нет.

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(8, 8))
sns.boxplot(y="street", x="number", data=data.query('street in @top_streets'), hue='chain', fliersize = 0).set(
    title='Количество посадочных мест на улицах с наибольшим количеством \n заведений общественного питания',
    xlabel='Количество посадочных мест',
    ylabel='Улица')
plt.xlim(-10, 280)

plt.show()

В сетевых заведениях количество посадочных мест, как правило, немного больше, чем в несетевых, за исключением улиц: Кутузовский проспект (возможно, это связано с тем, что район очень дорогой и все заведения ориентированы на состоятельных клииентов, не склонных посещать сетевые заведения) и Ленинградский проспект (тут разница совсем небольшая).  
Напротив, разброс посадочных мест в сетевых заведениях сильно больше, чем не в сетевых, на улицах: Каширское шоссе, Проспект Мира. Можно предположить, что в этих районах заведения больше ориентированы на менее обеспеченных посетителей.

Посмотрим, есть ли различия в количестве посадочных мест по округам:

In [ ]:
data = data.merge(data_street, how='left', on='street')

data.columns

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(y="district", x="number", data=data[data['n_districts'] == 1], fliersize = 3).set(
    title='Распределение количества посадочных мест по округам \n ',
    xlabel='Количество посадочных мест',
    ylabel='Округ')

plt.show()

Заведения с самым большим количеством посадочных мест находятся в ЗАО и ЦАО (и одно - в ВАО). Их очень мало, единицы. Можно предположить, что это дорогие и хорошо раскрученные рестораны с хорошей репутацией и постоянным потоком клиентов.  

Спрячем выбросы и добавим подкатегорию: сетевое заведение или нет.

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(y="district", x="number", data=data[data['n_districts'] == 1], hue='chain', fliersize = 0).set(
    title='Распределение количества посадочных мест по округам \n ',
    xlabel='Количество посадочных мест',
    ylabel='Округ')
plt.xlim(-10, 300)

plt.show()

Различий по округам практически нет. Разброс количества посадочных мест во всех округах больше в сетевых заведениях. В ЮВАО в сетевых заведениях несколько больше посадочных мест, чем в несетевых, а Зеленограде, наоборот, в несетевых немного больше, чем в сетевых.

#### Вывод


**Количество посадочных мест по округам**  
Различий в количестве посадочных мест по округам практически нет, а разброс во всех округах больше в сетевых заведениях.

**Количество посадочных мест на улицах с наибольшим количеством заведений**  
На популярных улицах в сетевых заведениях количество посадочных мест, как правило, немного больше, чем в несетевых, за исключением улиц: Кутузовский проспект (возможно, это связано с тем, что район очень дорогой и все заведения ориентированы на состоятельных клииентов, не склонных посещать сетевые заведения) и Ленинградский проспект (тут разница совсем небольшая).  
Разброс посадочных мест в сетевых заведениях сильно больше, чем не в сетевых, на улицах: Каширское шоссе, Проспект Мира. Можно предположить, что в этих районах заведения больше ориентированы на менее обеспеченных посетителей.

## Выводы и рекомендации

### Краткий обзор проведённой работы

Данные подготовлены к работе. Найдены и удалены дубликаты, выделены в отдельный столбец и приведены к единому формату данные об улицах. 

Проанализированы параметры заведений общественного питания:  
    - тип заведения
    - сетевое или несетевое заведение
    - количество посадочных мест

Загружены и подготовлены данные о районах и округах для каждой улицы.
    
Проанализировано расположение улиц с наибольшим количеством заведений общественного питания, сформирован список наиболее популярных улиц и наиболее популярных районов, построены графики. Проанализировано расположение улиц с одним заведением общественного питания.
Проанализировано распределение количества посадочных мест в заведениях общественного питания на наиболее популярных улицах и их распределение по типам заведений.
    
Сформулированы выводы и рекомендации.

### Выводы
#### **Выводы по продуктовым метрикам**

1. В Москве больше всего заведений типа кафе, меньше столовые, рестораны и предприятия быстрого обслуживания. Меньше всего по количеству буфетов, баров, закусочных и отделов кулинарий.  
    
    
2. В Москве преобладают несетевые заведения общественного питания. Сетевых заведений больше только среди предприятий быстрого обслуживания. Достаточно большая доля сетевых заведених среди кафе, ресторанов.
    
    
3. Больше всего посадочных мест обычно в столовых, ресторанах и сетевых барах.  
Меньше всего посадочных мест в закусочных, кафетериях и отделах кулинарий.  
В предприятиях быстрого обслуживания и барах в сетевых заведениях больше посадочных мест, чем в несетевых.  
В сетевых столовых и буфетах посадочных мест гораздо меньше, чем в несетевых. 
    
    
4. Больше всего заведений общественного питания расположено на магистральных улицах, ведущих из центра за пределы города: 
     - проспект Мира,
     - Варшавское шоссе
     - Ленинский проспект
     - Профсоюзная улица
     - Дмитровское шоссе
     - Ленинградский проспект
     - проспект Вернадского
     - Ленинградское шоссе
     - Каширское шоссе
     - Волгоградский проспект
     - Пресненская набережная
     - Кутузовский проспект
     - Кировоградская улица   
     
Большое количество заведений общественного питания на магистральных улицах (магистральные улицы остаются самыми популярными даже после коррекции на длину улицы). Скорее всего, это связано с тем, что на них находятся крупные транспортные узлы, через которые проходит много людей.  
На маленьких улицах, располагающихся в границах одного района, больше всего заведений общественного питания в центре (ЦАО).
    
    
5. Больше всего улиц с единственным заведением общественного питания находится в центре, однако доля таких улиц от общего количества улиц (на которых есть хотя бы одно заведение общественного питания) больше всего в периферийных спальных районах. При этом доля улиц с единственным заведением сильно различается по районами внутри одного округа. Вероятно, это связано с тем, что в спальных районах небольшой спрос на услуги кафе и ресторанов, и общее количество таких заведений небольшое, а ближе к транспортным узлам или другим точкам притяжения посетителей, спрос больше.  
    
    
6. Различий в количестве посадочных мест по округам практически нет, а разброс во всех округах больше в сетевых заведениях. 
На популярных улицах в сетевых заведениях количество посадочных мест, как правило, немного больше, чем в несетевых, за исключением улиц: Кутузовский проспект (возможно, это связано с тем, что район очень дорогой и все заведения ориентированы на состоятельных клииентов, не склонных посещать сетевые заведения) и Ленинградский проспект (тут разница совсем небольшая).   
Разброс посадочных мест в сетевых заведениях сильно больше, чем не в сетевых, на улицах: Каширское шоссе, Проспект Мира. Можно предположить, что в этих районах заведения больше ориентированы на менее обеспеченных посетителей.
    

### Рекомендации
При выборе места для заведения общественного питания стоит обратить внимание на магистральные улицы, соединяющие несколько районов. В случае, если заведение будет размещено на небольшой улице, лучше ограничить район выбора места центральным округом Москвы.  
Если заведение позиционируется как доступное для всех, можно выбрать сетевой формат кафе, бара, ресторана или предприятия быстрого обслуживания. В таком случае лучше выбирать расположение заведений не в спальных районах, близко к магистральным улицам и транспортным узнам, через которые ежедневно проходит много людей.  
Количество посадочных мест в любом случае стоит сделать небольшим (от 25 да 150 в зависимости от типа заведения). Для ресторана и сетевого бара количество посудочных мест может быть смещено в большую сторону, а для кафе и предприятия быстрого огбслуживания - в меньшую.